In [126]:
%matplotlib inline

import numpy as np
import pyvista as pv
from scipy.spatial import KDTree
from binary_to_csv import binary_to_csv
import os
import pandas as pd

In [127]:
ASSETS_DIR = '../assets'

grid_points = np.load(os.path.join(ASSETS_DIR, "boob_grid_12_21.npy"))

# Remove points that are outside of the boob mesh.
# Because those points have been skipped during the points mapping, their value is (0, 0, 0)
origin = np.array([0, 0, 0])
points_to_skip = np.where((grid_points == origin).all(axis=1))[0]
points_of_interest = np.delete(grid_points, points_to_skip, axis=0)

In [128]:
df_pre = pd.read_csv("../data/results/3-data_20241115_134123_pre.csv", header=None).drop(columns=points_to_skip)
df_post = pd.read_csv("../data/results/3-data_20241115_134545_post.csv", header=None).drop(columns=points_to_skip)

In [129]:
# Create the point cloud and initialize the scalars
point_cloud = pv.PolyData(points_of_interest)

p1 = pv.Plotter(border=False)
p1.add_title("Breast points", font_size=12)
p1.add_mesh(point_cloud, copy_mesh=True ,scalars=np.zeros(points_of_interest.shape[0]), cmap='cool', point_size=15)

p1.remove_scalar_bar()

p1.camera_position = 'xy'
p1.camera.elevation += 100

# p1.show()


In [130]:
K = 5

# Load the boob 3D model from file
boob_mesh = pv.read( '../assets/boob.obj')
boob_mesh.translate(np.array([0, -0.035, 0]), inplace=True)

# Create a kd-tree for quick nearest-neighbor lookup.
kdtree = KDTree(points_of_interest)

# Find the K nearest point_cloud points for each points in the boob mesh and calculate their respective distances
boob_mesh['nearest_points'] = kdtree.query(boob_mesh.points, k=K)[1]
boob_mesh['nearest_points_dist'] = kdtree.query(boob_mesh.points, k=K)[0]

# Step 1: Calculate the reciprocal of the distances
reciprocal_distances = 1 / boob_mesh['nearest_points_dist']

# Step 2: Square the reciprocal distances
squared_reciprocal_distances = reciprocal_distances ** 1.2


# Step 3: Normalize the weights
normalized_weights = squared_reciprocal_distances / \
    np.sum(squared_reciprocal_distances, axis=1, keepdims=True)

boob_mesh['pressure'] = np.zeros(boob_mesh.points.shape[0])
boob_mesh['max_pressure'] = boob_mesh['pressure']


p4 = pv.Plotter(border=False)
p4.add_title("Breast model", font_size=12)
p4.add_mesh(boob_mesh, copy_mesh=True , scalars=np.zeros(boob_mesh.points.shape[0]), cmap='cool', point_size=15)

p4.remove_scalar_bar()

p4.camera_position = 'xy'
p4.camera.elevation += 100

# p4.show()



In [131]:
def interpolate_grid_on_model(grid_scalar):
    return np.sum(grid_scalar[boob_mesh['nearest_points']] * normalized_weights, axis=1)

In [132]:
THRESHOLD = 100

calibration_data_pre = df_pre.iloc[:20].median()
calibration_data_post = df_post.iloc[:20].median()

points_touched_pre = (df_pre > calibration_data_pre + THRESHOLD).any().values
points_touched_post = (df_post > calibration_data_post + THRESHOLD).any().values

points_touched_interpolated_pre = np.sum(points_touched_pre[boob_mesh['nearest_points']] * normalized_weights, axis=1) > 0
points_touched_interpolated_post = np.sum(points_touched_post[boob_mesh['nearest_points']] * normalized_weights, axis=1) > 0

shape = (3, 2) # (rows, columns)
row_weights = [ 1, 1, 0.2]
col_weights = [1, 1]
groups = [
    (1, 0),  
    (2, np.s_[:]), 
]


p2 = pv.Plotter( border=False, shape=shape, row_weights=row_weights, col_weights=col_weights, groups=groups, window_size=[1000, 1000])

p2.subplot(2, 0)
mesh = pv.PolyData(np.zeros((2, 3)))
mesh['data'] = (False, True)

actor = p2.add_mesh(mesh, copy_mesh=True, scalars='data', cmap='cool', point_size=5,  show_scalar_bar=False)
actor.visibility = False


scalar_bar = p2.add_scalar_bar(height=0.8, width=0.8, position_x=0.1)


p2.subplot(0, 0)
p2.add_title("Pre Training", font_size=8)
p2.add_mesh(point_cloud, copy_mesh=True, scalars=points_touched_pre, cmap='cool', point_size=5,  show_scalar_bar=False)

p2.subplot(0, 1)
p2.add_title("Post Training", font_size=8)
p2.add_mesh(point_cloud, copy_mesh=True, scalars=points_touched_post, cmap='cool', point_size=5,  show_scalar_bar=False)

p2.subplot(1, 0)
p2.add_mesh(boob_mesh, copy_mesh=True, scalars=points_touched_interpolated_pre, cmap='cool',  show_scalar_bar=False)


p2.subplot(1, 1)
p2.add_mesh(boob_mesh, copy_mesh=True, scalars=points_touched_interpolated_post, cmap='cool',  show_scalar_bar=False)

p2.link_views()

p2.camera_position = 'xy'
p2.camera.elevation += 100

p2.show()


# p2 = pv.Plotter( border=False, shape=(2, 2))

# p2.subplot(0, 0)
# p2.add_title("Points pressed", font_size=8)
# p2.add_mesh(point_cloud, copy_mesh=True, scalars=points_touched_pre, cmap='cool', point_size=10)

# p2.subplot(0, 1)
# p2.add_title("Points pressed", font_size=8)
# p2.add_mesh(point_cloud, copy_mesh=True, scalars=points_touched_post, cmap='cool', point_size=10)

# p2.subplot(1, 0)
# p2.add_title("Points pressed", font_size=8)
# p2.add_mesh(boob_mesh, copy_mesh=True, scalars=points_touched_interpolated_pre, cmap='cool', point_size=10)

# p2.subplot(1, 1)
# p2.add_title("Points pressed", font_size=8)
# p2.add_mesh(boob_mesh, copy_mesh=True, scalars=points_touched_interpolated_post, cmap='cool', point_size=10)

# p2.link_views()

# p2.camera_position = 'xy'
# p2.camera.elevation += 100



# p2.show()

Widget(value='<iframe src="http://localhost:61568/index.html?ui=P_0x31eb174d0_105&reconnect=auto" class="pyvis…

 
## Visualization of Points Touched Before and After Training


In [133]:
n_touch_pre = (df_pre > calibration_data_pre + THRESHOLD).sum(axis=0)
n_touch_post = (df_post > calibration_data_post + THRESHOLD).sum(axis=0)

n_touch_interpolated_pre = np.sum(n_touch_pre.values[boob_mesh['nearest_points']] * normalized_weights, axis=1)
n_touch_interpolated_post = np.sum(n_touch_post.values[boob_mesh['nearest_points']] * normalized_weights, axis=1)


shape = (3, 2) # (rows, columns)
row_weights = [ 1, 1, 0.2]
col_weights = [1, 1]
groups = [
    (1, 0),  
    (2, np.s_[:]), 
]


p3 = pv.Plotter( border=False, shape=shape, row_weights=row_weights, col_weights=col_weights, groups=groups, window_size=[1000, 1000])

p3.subplot(2, 0)
mesh = pv.PolyData(np.zeros((2, 3)))
mesh['data'] = (0, 1)

actor = p3.add_mesh(point_cloud, copy_mesh=True, scalars=n_touch_pre, cmap='cool', point_size=5,  show_scalar_bar=False)
actor.visibility = False


scalar_bar = p3.add_scalar_bar(height=0.5, width=0.8, position_x=0.1, title="Number of Point Activations")

p3.subplot(0, 0)
p3.add_title("Pre Training", font_size=8)
p3.add_mesh(point_cloud, copy_mesh=True, scalars=n_touch_pre, cmap='cool', point_size=5,  show_scalar_bar=False)

p3.subplot(0, 1)
p3.add_title("Post Training", font_size=8)
p3.add_mesh(point_cloud, copy_mesh=True, scalars=n_touch_post, cmap='cool', point_size=5,  show_scalar_bar=False)

p3.subplot(1, 0)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=n_touch_interpolated_pre, cmap='cool',  show_scalar_bar=False)


p3.subplot(1, 1)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=n_touch_interpolated_post, cmap='cool',  show_scalar_bar=False)

p3.link_views()

p3.camera_position = 'xy'
p3.camera.elevation += 100

p3.show()

Widget(value='<iframe src="http://localhost:61568/index.html?ui=P_0x31eb17500_106&reconnect=auto" class="pyvis…

In [134]:

# relative_pressure = df_poi - calibration_data



# p3 = pv.Plotter(border=False, shape=(2, 2))

# p3.subplot(0, 0)
# p3.add_title("Mean Pressure", font_size=8)
# p3.add_mesh(point_cloud, copy_mesh=True, scalars=relative_pressure.mean(), scalar_bar_args={
#              'title': 'mean', "title_font_size": -1}, cmap='cool', point_size=10)

# p3.subplot(1, 0)
# p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(relative_pressure.mean().values), scalar_bar_args={
#              'title': 'mean', "title_font_size": -1}, cmap='cool', point_size=10)

# p3.subplot(0, 1)
# p3.add_title("Max Pressure", font_size=8)
# p3.add_mesh(point_cloud, copy_mesh=True, scalars=relative_pressure.max(), scalar_bar_args={
#              'title': 'max', "title_font_size": -1}, cmap='cool', point_size=10)

# p3.subplot(1, 1)
# p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(relative_pressure.max().values), scalar_bar_args={
#              'title': 'max', "title_font_size": -1}, cmap='cool', point_size=10)

# p3.link_views()


# p3.camera_position = 'xy'
# p3.camera.elevation += 100

# p3.show(jupyter_backend='trame')

